In [1]:
import pandas as pd
import yfinance as yf
import sqlalchemy as db
import pyodbc

In [2]:
tickers = ['JOE', 'NEM', 'ASC', 'INVA', 'TRIP', 'JBLU', 'GEN', 'PLTR', 'ACHR', 'BAC']
start_date = '2022-01-01'

data = yf.download(tickers, start=start_date, interval='1d', group_by='ticker', auto_adjust=True, actions=True)

# Concatenate all the dataframes into one
df = pd.concat([data[ticker] for ticker in tickers], keys=tickers)

# Reset index to get 'Date' as a column instead of index
df.reset_index(inplace=True)

# Rename the 'level_0' column to 'Ticker'
df.rename(columns={'level_0': 'Ticker'}, inplace=True)

# Reorder columns 
df = df[['Ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']]

print(df)


[*********************100%***********************]  10 of 10 completed
     Ticker       Date       Open       High        Low      Close    Volume  \
0       JOE 2022-01-03  51.258678  52.120989  50.817721  51.141087    102400   
1       JOE 2022-01-04  51.474258  53.796629  51.052900  53.375271    256500   
2       JOE 2022-01-05  53.679042  54.374775  51.150896  51.444866    162500   
3       JOE 2022-01-06  51.680047  52.620752  51.052906  52.111202     99600   
4       JOE 2022-01-07  51.905418  52.532555  50.817726  51.376270    119000   
...     ...        ...        ...        ...        ...        ...       ...   
5715    BAC 2024-04-08  37.200001  37.580002  37.130001  37.500000  29470300   
5716    BAC 2024-04-09  37.509998  37.770000  37.200001  37.730000  30977700   
5717    BAC 2024-04-10  37.180000  37.230000  36.549999  36.650002  37610700   
5718    BAC 2024-04-11  36.750000  36.750000  36.049999  36.349998  38604000   
5719    BAC 2024-04-12  35.810001  36.090000  35.

In [3]:
df.rename(columns={'Stock Splits': 'StockSplits'}, inplace=True)
df

,Ticker,Date,Open,High,Low,Close,Volume,Dividends,StockSplits
0,JOE,2022-01-03,51.258678,52.120989,50.817721,51.141087,102400,0.0,0
1,JOE,2022-01-04,51.474258,53.796629,51.052900,53.375271,256500,0.0,0
2,JOE,2022-01-05,53.679042,54.374775,51.150896,51.444866,162500,0.0,0
3,JOE,2022-01-06,51.680047,52.620752,51.052906,52.111202,99600,0.0,0
4,JOE,2022-01-07,51.905418,52.532555,50.817726,51.376270,119000,0.0,0
...,...,...,...,...,...,...,...,...,...
5715,BAC,2024-04-08,37.200001,37.580002,37.130001,37.500000,29470300,0.0,0
5716,BAC,2024-04-09,37.509998,37.770000,37.200001,37.730000,30977700,0.0,0
5717,BAC,2024-04-10,37.180000,37.230000,36.549999,36.650002,37610700,0.0,0
5718,BAC,2024-04-11,36.750000,36.750000,36.049999,36.349998,38604000,0.0,0


In [4]:
from sqlalchemy import create_engine
import urllib

# Your connection parameters are formatted into a URL
params = urllib.parse.quote_plus(
    "Driver={SQL Server};"
    "Server=;"  # Make sure 'server' is replaced by your actual server name
    "Database=;"
    "Trusted_Connection=yes;"
)

# Create an SQLAlchemy engine using the connection URL
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [5]:
df.to_sql('Stocks', con=engine, if_exists='append', index=False)

In [7]:
df.to_csv("df.csv")